In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import chdir
from warnings import filterwarnings

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
# from sklearn.preprocessing import StandardScaler (não vou importar porque vamos utilizar um tree-based model)

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.ensemble import RandomForestClassifier

chdir(r'C:\Users\Anwar\Desktop\VSCode\ML Olympiad\Dados')
filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
df.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,Negado
1,EZYV5505,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,Aprovado
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,Negado
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,Negado
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,Aprovado


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17836 entries, 0 to 17835
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_do_caso                      17836 non-null  object 
 1   continente                      17836 non-null  object 
 2   educacao_do_empregado           17836 non-null  object 
 3   tem_experiencia_de_trabalho     17836 non-null  object 
 4   requer_treinamento_de_trabalho  17836 non-null  object 
 5   num_de_empregados               17836 non-null  int64  
 6   ano_de_estabelecimento          17836 non-null  int64  
 7   regiao_de_emprego               17836 non-null  object 
 8   salario_prevalecente            17836 non-null  float64
 9   unidade_de_salario              17836 non-null  object 
 10  posicao_em_tempo_integral       17836 non-null  object 
 11  status_do_caso                  17836 non-null  object 
dtypes: float64(1), int64(2), object(

In [5]:
df.shape

(17836, 12)

In [10]:
df.tem_experiencia_de_trabalho = df.tem_experiencia_de_trabalho.apply(lambda x: 1 if x == 'S' else 0)
df.requer_treinamento_de_trabalho = df.requer_treinamento_de_trabalho.apply(lambda x: 1 if x == 'S' else 0)
df.posicao_em_tempo_integral = df.posicao_em_tempo_integral.apply(lambda x: 1 if x == 'S' else 0)
df.status_do_caso = df.status_do_caso.apply(lambda x: 1 if x == 'Aprovado' else 0)

In [11]:
df.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,0,1,2087,1855,Sul,69711.24,Ano,1,0
1,EZYV5505,Ásia,Mestrado,1,0,5991,2003,Meio-Oeste,52931.38,Ano,1,1
2,EZYV5207,Ásia,Ensino Médio,0,0,1426,2000,Ilha,110830.21,Ano,1,0
3,EZYV7545,Ásia,Ensino Médio,0,0,3846,1992,Meio-Oeste,91884.68,Semana,1,0
4,EZYV16071,Ásia,Ensino Superior,1,0,3957,1949,Nordeste,138155.24,Ano,1,1


In [9]:
df.status_do_caso.value_counts(1)

Aprovado    0.669264
Negado      0.330736
Name: status_do_caso, dtype: float64

In [41]:
df.isnull().mean()

id_do_caso                        0.0
continente                        0.0
educacao_do_empregado             0.0
tem_experiencia_de_trabalho       0.0
requer_treinamento_de_trabalho    0.0
num_de_empregados                 0.0
ano_de_estabelecimento            0.0
regiao_de_emprego                 0.0
salario_prevalecente              0.0
unidade_de_salario                0.0
posicao_em_tempo_integral         0.0
status_do_caso                    0.0
dtype: float64

## Modelagem

In [15]:
X = df.drop(columns = ['id_do_caso', 'status_do_caso'], axis = 1)
y = df.status_do_caso

In [26]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [31]:
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 10)

In [52]:
cat_cols = X.select_dtypes(include = ['object']).columns
num_cols = X.select_dtypes(exclude = ['object']).columns

In [95]:
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')), ('encoder', TargetEncoder()), ('imputer2', SimpleImputer(strategy = 'most_frequent'))])
num_pipe = Pipeline([('imputer', SimpleImputer(strategy = 'median'))])

In [100]:
def cross_validation(X, y, cat_pipe, num_pipe, modelo, threshold = 0.5):
    f1_list = list()
    roc_list = list()
    for i, (index_train, index_valid) in enumerate(folds.split(X, y)):
        print(f"-=-=-= FOLD {i+1} -=-=-=")
        X_train_interno, y_train_interno = X.iloc[index_train, :], y[index_train]
        X_valid_interno, y_valid_interno = X.iloc[index_valid, :], y[index_valid]
        
        # Fazer uma feature engineering
        
        
        # Preprocessamento
        X_train_interno[cat_cols] = cat_pipe.fit_transform(X_train_interno[cat_cols], y_train_interno)
        X_valid_interno[cat_cols] = cat_pipe.transform(X_valid_interno[cat_cols])
        
        X_train_interno[num_cols] = num_pipe.fit_transform(X_train_interno[num_cols])
        X_valid_interno[num_cols] = num_pipe.transform(X_valid_interno[num_cols])
        
        # Treinar o modelo
        modelo.fit(X_train_interno, y_train_interno)
        y_pred_proba = modelo.predict_proba(X_valid_interno)[:, 1]
        y_pred = np.where(y_pred_proba > threshold, 1, 0)
        
        # Calcular as métricas
        valor_f1 = f1_score(y_valid_interno, y_pred)
        valor_roc = roc_auc_score(y_valid_interno, y_pred_proba)
        f1_list.append(valor_f1)
        roc_list.append(valor_roc)
        print(f"F1: {valor_f1:.4f}\nROC: {valor_roc:.4f}")
        print('')
    
    # Métricas finais
    print(f"F1: {np.mean(f1_list)} +- {np.std(f1_list)}")
    print(f"ROC: {np.mean(roc_list)} +- {np.std(roc_list)}")

In [97]:
modelo = RandomForestClassifier(n_estimators = 1000, n_jobs = -1, random_state = 0, class_weight = {1: 1, 0: 2.02355957621789})

In [99]:
cross_validation(X, y, cat_pipe, num_pipe, modelo, threshold = 0.67)

-=-=-= FOLD 0 -=-=-=
F1: 0.7531
ROC: 0.7700

-=-=-= FOLD 1 -=-=-=
F1: 0.7485
ROC: 0.7484

-=-=-= FOLD 2 -=-=-=
F1: 0.7486
ROC: 0.7621

-=-=-= FOLD 3 -=-=-=
F1: 0.7619
ROC: 0.7594

-=-=-= FOLD 4 -=-=-=
F1: 0.7596
ROC: 0.7636

F1: 0.7543613098768245 +- 0.00552660330857269
ROC: 0.760695524842631 +- 0.00709519162814482


## Resultado para threshold = 0.5, random_forest(n_estimators = 1000)
F1: 0.8093054599787047 +- 0.004550924906060425

ROC: 0.760695524842631 +- 0.00709519162814482

## Resultado para threshold = 0.67, random_forest(n_estimators = 1000)
F1: 0.7543613098768245 +- 0.00552660330857269

ROC: 0.760695524842631 +- 0.00709519162814482